<a href="https://colab.research.google.com/github/CarolineKahalf/Space-Invaders/blob/main/Space_Invaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

     |████████████████████████████████| 320.4 MB 25 kB/s 
     |████████████████████████████████| 52 kB 674 kB/s 
     |████████████████████████████████| 20.1 MB 134 kB/s 
     |████████████████████████████████| 2.9 MB 60.9 MB/s 
     |████████████████████████████████| 459 kB 41.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installatio

In [ ]:
import gym 
import random

In [ ]:
# import ROMs for space invaders
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
mkdir: cannot create directory ‘rars’: File exists
copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_

In [ ]:
env = gym.make('SpaceInvaders-v0') 
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:65.0
Episode:2 Score:120.0
Episode:3 Score:45.0
Episode:4 Score:110.0
Episode:5 Score:145.0


In [ ]:
### Creating a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
del model

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_9 (Dense)              (None, 6)                

In [ ]:
 ### Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  821/10000: episode: 1, duration: 25.459s, episode steps: 821, steps per second:  32, episode reward: 185.000, mean reward:  0.225 [ 0.000, 30.000], mean action: 2.470 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 1797/10000: episode: 2, duration: 1453.814s, episode steps: 976, steps per second:   1, episode reward: 440.000, mean reward:  0.451 [ 0.000, 200.000], mean action: 2.508 [0.000, 5.000],  loss: 12.743277, mean_q: 9.256041, mean_eps: 0.874135
 2585/10000: episode: 3, duration: 1433.816s, episode steps: 788, steps per second:   1, episode reward: 410.000, mean reward:  0.520 [ 0.000, 200.000], mean action: 2.627 [0.000, 5.000],  loss: 5.458917, mean_q: 11.569056, mean_eps: 0.802855
 3155/10000: episode: 4, duration: 1042.761s, episode steps: 570, steps per second:   1, episode reward: 110.000, mean reward:  0.193 [ 0.000, 30.000], mean action: 2.404 [0.000, 5.000],  loss: 1.404372, mean_q: 10.962948, mean_eps: 0.741745
 3616/10000: episode: 5, duration: 842.726s, episode steps: 461, steps per second:   1, episode reward: 30.000, mean reward:  0.065 [ 0.000, 15.000], mean action: 2.679 [0.000, 5.000],  loss: 0.601769, mean_q: 8.883481, mean_eps: 0.695350
 4668/10000: episode: 6, duration

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 465.000, steps: 1026
Episode 2: reward: 270.000, steps: 733
Episode 3: reward: 280.000, steps: 725
Episode 4: reward: 280.000, steps: 726
Episode 5: reward: 270.000, steps: 724
Episode 6: reward: 270.000, steps: 712
Episode 7: reward: 465.000, steps: 1056
Episode 8: reward: 325.000, steps: 712
Episode 9: reward: 310.000, steps: 706
Episode 10: reward: 280.000, steps: 719
321.5
